In [8]:
#%load_ext autoreload
#%autoreload 2

#import sys
#sys.path.insert(0,'..')

import os

import backtrader as bt
import backtrader.indicators as btind
import numpy as np
import scipy.signal as signal
from scipy import stats

from gym import spaces

from btgym import BTgymEnv, BTgymStrategy, BTgymDataset

from launcher import Launcher

In [4]:
# GYM TEST ENV:
cluster_config = dict(
    host='127.0.0.1',
    port=12222,
    num_workers=8,
    num_ps=1,
    log_dir='./tmp/a3c_testing_gym',
)

env_config = dict(
    gym_id='Breakout-v0'
)


launcher = Launcher(
    cluster_config=cluster_config,
    env_config=env_config,
    train_steps=500000000,
    opt_learn_rate=1e-4,
    rollout_length=30,
    test_mode=True,
    model_summary_freq=50,
    episode_summary_freq=2,
    env_render_freq=10,
    verbose=1
)

[2017-08-22 14:36:57,411] ./tmp/a3c_testing_gym created.


In [5]:
launcher.run()


[2017-08-22 14:37:00,994] Making new env: Breakout-v0
[2017-08-22 14:37:06,085] Press `Ctrl-C` to stop training and close launcher.
[2017-08-22 14:37:06,047] Making new env: Breakout-v0
[2017-08-22 14:37:06,136] Making new env: Breakout-v0
[2017-08-22 14:37:06,152] Making new env: Breakout-v0
[2017-08-22 14:37:06,141] Making new env: Breakout-v0
[2017-08-22 14:37:06,144] Making new env: Breakout-v0
[2017-08-22 14:37:06,186] Making new env: Breakout-v0
[2017-08-22 14:37:06,198] Making new env: Breakout-v0


Press `Ctrl-C` to stop training and close launcher.
INFO:tensorflow:Starting standard services.


[2017-08-22 14:37:11,444] Starting standard services.


INFO:tensorflow:Saving checkpoint to path ./tmp/a3c_testing_gym/train/model.ckpt


[2017-08-22 14:37:11,464] Saving checkpoint to path ./tmp/a3c_testing_gym/train/model.ckpt


...

[2017-08-22 14:37:15,270] Starting queue runners.
[2017-08-22 14:37:15,318] worker_4: starting training at step: 344
[2017-08-22 14:37:15,365] worker_3: starting training at step: 344


INFO:tensorflow:Starting queue runners.


[2017-08-22 14:37:15,425] Starting queue runners.
[2017-08-22 14:37:15,485] worker_5: starting training at step: 344


INFO:tensorflow:global/global_step/sec: 522.932


[2017-08-22 14:39:11,477] global/global_step/sec: 522.932
[2017-08-22 14:40:43,043] worker_1 has joined.
[2017-08-22 14:40:43,049] worker_2 has joined.
[2017-08-22 14:40:43,050] worker_3 has joined.
[2017-08-22 14:40:43,051] worker_4 has joined.
[2017-08-22 14:40:43,052] worker_5 has joined.
[2017-08-22 14:40:43,053] worker_6 has joined.
[2017-08-22 14:40:43,054] worker_7 has joined.
[2017-08-22 14:40:43,055] chief_worker_0 has joined.
[2017-08-22 14:40:43,055] parameter_server_0 has joined.
[2017-08-22 14:40:43,056] Launcher closed.


In [13]:
class MyStrategy(BTgymStrategy):
    """
    Example subclass of BT server inner computation startegy.
    """
    
    def __init__(self, **kwargs):
        super(MyStrategy,self).__init__(**kwargs)
        
        self.order_penalty = 0.5
        self.trade_just_closed = False
        self.trade_result = None
        
    def notify_trade(self, trade):
        #if trade.justopened:
            #print('trade {} just opened'.format(trade.ref))
            
        if trade.isclosed:
            #print('trade {} closed, pnl_comm: {}'.format(trade.ref, trade.pnlcomm))
            # Set trade flag and result:
            self.trade_just_closed = True
            self.trade_result = trade.pnlcomm
    
    def sigmoid(self, x):
        return 1/(1 + np.exp(-x))
        
    def get_state(self):
        """
        Computes featurized RL-ready environment observation state
        by applying continious wavelet transform to time-embedded vector
        of close-price gradients.
        """
        # Use close price:
        channel = 3
        
        X = self.raw_state[:, channel]
        
        # Prepare parameters:
        Tau = 5
        max_cwt_scale = self.p.state_shape['model_input'].shape[1]
        cwt_width = np.linspace(Tau, max_cwt_scale + Tau - 1, max_cwt_scale) # scale of wavelet transdorm [n]
    
        T = 1# 000
        
        # Get vector of gradients of last [n] prices:
        X = np.gradient(X, axis=0) * T
        
        # Compute continious wavelet transform using Ricker wavelet, get [n,m,1]-dim. matrix:
        X = signal.cwt(X, signal.ricker, cwt_width).T[:, :, None]
        
        # Z-score:
        X = stats.zscore(X)
        
        # Local min-max norm:
        self.state['model_input'] = (X - X.min()) / (X.max() - X.min())
        
        # Squash values in [0,1]:
        #self.state['model_input'] = self.sigmoid(self.state['model_input'])
        
        return self.state

    
    def get_reward(self):
        """
        Defines reward as [0,1]-bounded function of last closed trade result.
        """
        r = 0
        
        # Result
        if self.trade_just_closed:
            r = self.trade_result
            self.trade_just_closed = False
            
        # Penalty for failed order:
        if self.order_failed:
            #print('Failed order!')
            r -= self.order_penalty
            self.order_failed = False
            
        #print('reward_', r)
        
        return r / 20
            
# Set backtesting engine parameters:

state_shape = {
    'raw_state': spaces.Box(low=-1, high=1, shape=(120, 4)),
    'model_input': spaces.Box(low=-10, high=10, shape=(120, 15, 1))
}

MyCerebro = bt.Cerebro()
MyCerebro.addstrategy(MyStrategy,
                      state_shape=state_shape,
                      portfolio_actions=('hold', 'buy', 'sell'),
                      drawdown_call=5, # in percent
                      target_call=10,
                      skip_frame=10,
                     )

# Set leveraged account:
MyCerebro.broker.setcash(2000)
MyCerebro.broker.setcommission(commission=0.0001, leverage=10.0)
MyCerebro.broker.set_shortcash(False)
MyCerebro.addsizer(bt.sizers.SizerFix, stake=10000,)


MyCerebro.addanalyzer(bt.analyzers.DrawDown)

# Provide data:
MyDataset = BTgymDataset(
    filename='../data/DAT_ASCII_EURUSD_M1_2016.csv',
    #filename='../data/test_sine_1min_period256_delta0002.csv',
    start_weekdays=[0, 1, 2, 3, 4],
    episode_len_days=0,
    episode_len_hours=23,
    episode_len_minutes=0,
    start_00=True,
    time_gap_hours=2,
)
env_config = dict(
    dataset=MyDataset,
    engine=MyCerebro,
    render_modes=['episode', 'human', 'model_input'],
    render_state_as_image=True,
    render_ylabel='Z-norm/CWT',
    render_size_episode=(12,8),
    render_size_human=(8, 3.5),
    render_size_state=(10, 5),
    render_dpi=75,
    port=5000,
    data_port=4999,
    connect_timeout=60,
    verbose=0,
)
cluster_config = dict(
    host='127.0.0.1',
    port=12222,
    num_workers=8,
    num_ps=1,
    log_dir='./tmp/a3c_testing',
)
launcher = Launcher(
    cluster_config=cluster_config,
    env_class=BTgymEnv,
    env_config=env_config,
    rollout_length=20,
    test_mode=False,
    train_steps=1000000000,
    model_summary_freq=20,
    episode_summary_freq=1,
    env_render_freq=20,
    verbose=1
    
)

In [14]:
launcher.run()

[2017-08-23 11:42:52,680] Press `Ctrl-C` to stop training and close launcher.


Press `Ctrl-C` to stop training and close launcher.
INFO:tensorflow:Restoring parameters from ./tmp/a3c_testing/train/model.ckpt-17976671


[2017-08-23 11:43:00,619] Restoring parameters from ./tmp/a3c_testing/train/model.ckpt-17976671


INFO:tensorflow:Starting queue runners.


[2017-08-23 11:43:02,247] Starting queue runners.


...

[2017-08-23 15:08:05,844] Saving checkpoint to path ./tmp/a3c_testing/train/model.ckpt


INFO:tensorflow:Saving checkpoint to path ./tmp/a3c_testing/train/model.ckpt


[2017-08-23 15:13:05,839] Saving checkpoint to path ./tmp/a3c_testing/train/model.ckpt
Process BTgymServer-54:1:
Process BTgymServer-52:1:
Traceback (most recent call last):
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
[2017-08-23 15:16:44,334] worker_1 has joined.
  File "/Users/muzikin/Yandex.Disk.localized/work/btgym/btgym/server.py", line 413, in run
    episode = cerebro.run(stdstats=True, preload=False, oldbuysell=True)[0]
Traceback (most recent call last):
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/backtrader/cerebro.py", line 1073, in run
    runstrat = self.runstrategies(iterstrat)
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/backtrader/cerebro.py", line 1235, in runstrategies
    self._runnext(runstrats)
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/backtrader/cerebro.py", line 1491, in _runnext


  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/backtrader/cerebro.py", line 1564, in _runnext
    strat._next()
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/backtrader/strategy.py", line 361, in _next_analyzers
    analyzer._next()
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/backtrader/strategy.py", line 327, in _next
    self._next_analyzers(minperstatus)
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/backtrader/strategy.py", line 327, in _next
    self._next_analyzers(minperstatus)
  File "/Users/muzikin/Yandex.Disk.localized/work/btgym/btgym/dataserver.py", line 94, in run
    episode_stat=episode_dataset.describe(),
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/backtrader/analyzer.py", line 188, in _next
    self.next()
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/backtrader/analyzer.py", line 188

  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/pandas/core/internals.py", line 1042, in take_nd
    allow_fill=False)
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/pandas/core/algorithms.py", line 1471, in take_nd
    func(arr, indexer, out, fill_value)
KeyboardInterrupt


In [ ]:
print(launcher.kwargs, '\n\n')
print(launcher.env_config)
print(launcher.cluster_config)
print(launcher.cluster_spec)
for config in launcher.workers_config_list:
    print('============')
    for k, v in config.items():
        print('{}:\n{}\n'.format(k, v))
    

In [ ]:
def func1(max_step):
    step = 0
    done = False
    
    def func2(max_step):
        nonlocal step
        nonlocal done
        step +=1
        if step == max_step:
            step = 0
            done = True
        return step
    
    for i in range(20):
        done = False
        print(func2(max_step), step, done)
        


func1(7)
            

In [ ]:
a = dict()
a.update({'b': 2, 'c':4})
type(a) == dict